Loading data:

In [178]:
import numpy as np
import pandas as pd
import json

with open("proj5_params.json", "r") as file:
    params = json.loads(file.read())

timeseries = pd.read_csv("proj5_timeseries.csv")

In [179]:
import re

for name in timeseries.columns:
    new_columnname = re.sub(r"[^0-9a-zA-Z]", "_", name)
    new_columnname = new_columnname.lower()
    timeseries.rename(columns={name : new_columnname}, inplace=True)

first_col = timeseries.columns[0]
timeseries[first_col] = pd.to_datetime(timeseries[first_col], format="mixed")
timeseries.set_index(first_col, inplace=True)
timeseries = timeseries.asfreq(params["original_frequency"]) 

In [180]:
import pickle

with open("proj5_ex01.pkl", "wb") as file:
    pickle.dump(timeseries, file)

In [181]:
timeseries_2 = timeseries.asfreq(params["target_frequency"]) 

with open("proj5_ex02.pkl", "wb") as file:
    pickle.dump(timeseries_2, file)

In [182]:
downsample_periods = params["downsample_periods"]
downsample_units = params["downsample_units"]
period_str = f"{downsample_periods}{downsample_units}"

count_per_window = timeseries.resample(period_str).count()
expected_count = downsample_periods

downsampled = timeseries.resample(period_str).sum()

downsampled[count_per_window < expected_count] = float('nan')

downsampled.to_pickle("proj5_ex03.pkl")

In [183]:
try:
    import scipy
except Exception:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scipy"])
    import scipy

In [184]:
rule = f'{params["upsample_periods"]}{params["upsample_units"]}'

upsampled = timeseries.resample(rule).asfreq()

upsampled_interpolated = upsampled.interpolate(
    method=params["interpolation"],
    order=params["interpolation_order"] if params["interpolation"] == "polynomial" else None
)

original_td = pd.Timedelta(1, unit=params["original_frequency"])
new_td = pd.Timedelta(params["upsample_periods"], unit=params["upsample_units"])
scaling_factor = new_td / original_td

numeric_cols = upsampled_interpolated.select_dtypes(include='number').columns
upsampled_interpolated[numeric_cols] *= scaling_factor

upsampled_interpolated.to_pickle("proj5_ex04.pkl")

In [ ]:
with open("proj5_sensors.pkl", "rb") as file:
    cos = pickle.load(file)

df = pd.DataFrame(cos)

if df.index.name != "timestamp":
    if "timestamp" in df.columns:
        df = df.set_index("timestamp")

wide_df = df.pivot(columns="device_id", values="value")

freq_str = f'{params["sensors_periods"]}{params["sensors_units"]}'
full_range = pd.date_range(start=wide_df.index.min(), end=wide_df.index.max(), freq=freq_str)

wide_df = wide_df.reindex(full_range)

wide_df.index.name = "timestamp"

wide_df.index.freq = pd.tseries.frequencies.to_offset(freq_str)

wide_df = wide_df.interpolate(method="linear", axis=0)

wide_df = wide_df.dropna(how='any', axis=0)
wide_df

# wide_df.to_pickle("proj5_ex05.pkl")


C:\Users\User\AppData\Local\Temp\ipykernel_49300\3715224482.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  cos = pickle.load(file)


device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-11-25 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,6.800000,NaN
2022-11-25 00:00:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.383333,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,6.795614,NaN
2022-11-25 00:00:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.369444,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,6.791228,NaN
2022-11-25 00:00:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.355556,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,6.786842,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.887255,...,NaN,4.515789,NaN,NaN,NaN,NaN,NaN,NaN,4.005882,NaN
2022-11-27 23:59:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.888235,...,NaN,4.515789,NaN,NaN,NaN,NaN,NaN,NaN,4.005882,NaN
2022-11-27 23:59:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.888235,...,NaN,4.515789,NaN,NaN,NaN,NaN,NaN,NaN,4.005882,NaN
